[App](https://quati-ana-clara.streamlit.app/)

In [6]:
# %pip install pytesseract pdf2image Pillow
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 42.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import cv2
import pytesseract
from pdf2image import convert_from_path
import PIL
from PIL import Image
import os
import re

os.getcwd()

'/Users/danusioguimaraes/Downloads/QUATI'

In [2]:
def preprocess_image(image):
    # Convertendo a imagem para numpy array (necessário para usar OpenCV)
    img_np = np.array(image)

    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    
    # Aplicar binarização (usando a técnica Otsu)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Converter de volta para imagem PIL
    return Image.fromarray(thresh)

In [10]:
list_prefix = ['ago. 4, Doc '+str(i) for i in range(2,9)]
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'
poppler_path = r'/opt/homebrew/bin/'
PIL.Image.MAX_IMAGE_PIXELS = None
custom_config = r'--oem 3 --psm 6'

text_extracted = ""
dict_heads = {}
dict_bodies = {}
for prefix in list_prefix:
    # heads
    try:
        image_path = 'Heads/' + prefix + ' head.png'
        image = Image.open(image_path)

        processed_img = preprocess_image(image)

        text = pytesseract.image_to_string(processed_img, lang='por', config=custom_config)
        
    except:
        text = f"{prefix + ' head.png'} not found."
        print(text)
    
    dict_heads[prefix] = text

    # bodies
    try:
        image_path = 'Bodies/' + prefix + ' body.png'
        image = Image.open(image_path)

        processed_img = preprocess_image(image)

        text = pytesseract.image_to_string(processed_img, lang='por', config=custom_config)

    except:
        msg = f"{prefix + ' body.png'} not found."
        print(msg)
        text = ''
    
    dict_bodies[prefix] = text

    print(f"Text for {prefix} is generated.")
    
for prefix in list_prefix:
    text_extracted += f"\n\n{dict_heads[prefix]} ---\n\n{dict_bodies[prefix]}"
    print(f"Text for {prefix} added to final file.")

text_extracted += '\nEND'

ago. 4, Doc 2 body.png not found.
Text for ago. 4, Doc 2 is generated.
Text for ago. 4, Doc 3 is generated.
Text for ago. 4, Doc 4 is generated.
Text for ago. 4, Doc 5 is generated.
Text for ago. 4, Doc 6 is generated.
Text for ago. 4, Doc 7 is generated.
Text for ago. 4, Doc 8 is generated.
Text for ago. 4, Doc 2 added to final file.
Text for ago. 4, Doc 3 added to final file.
Text for ago. 4, Doc 4 added to final file.
Text for ago. 4, Doc 5 added to final file.
Text for ago. 4, Doc 6 added to final file.
Text for ago. 4, Doc 7 added to final file.
Text for ago. 4, Doc 8 added to final file.


In [11]:
with open('texto_extraido.txt', 'w') as arquivo_texto:
    arquivo_texto.write(text_extracted)

print("Texto salvo com sucesso.")

Texto salvo com sucesso.


# Output Processing

In [25]:
with open('texto_extraido - v1.txt', 'r') as file:
    text_content = file.read()

print(text_content[100:200])

spostas que você já recebeu
e deve ter preenchido corretamente com todos os dados que lhe são pedido


In [13]:
def extract_substring(text, start_string, end_string):
    start_index = text.find(start_string)
    if start_index == -1:
        return "Start string not found"
    
    start_index += len(start_string)
    
    end_index = text.find(end_string, start_index)
    if end_index == -1:
        return "End string not found"
    
    return text[start_index:end_index]

text = "This is a sample text with a [start] target substring [end] that we want to extract."
start = "[start]"
end = "[end]"

substring = extract_substring(text, start, end)
print(substring)

 target substring 


In [14]:
starts = ['INSTRUÇÕES', 'A FESTA', 'O TRABALHO', 'A VIAGEM', 'VIDA PESSOAL', 'O LAZER', 'O ESTUDO']
ends = ['A FESTA', 'O TRABALHO', 'A VIAGEM', 'VIDA PESSOAL', 'O LAZER', 'O ESTUDO', 'END']

print(extract_substring(text_content, starts[4], ends[4]))



Observando Seus gostos e preferências pessoais, aquelas pequenas coisas que representam o seu Jeito particular de ser, 
quais seriam suas escolhas mais verdadeiras. Você se considera mais...
 ---

1. a - comunicativo.
b - observador.
2. a - teórico.
b - prático.
3. a- racional.
b - afetivo.
4. a - expansivo.
b - reservado.
5. a- planejador.
b - executor.
6. a- objetivo.
b - simpático.
7. a - dinâmico.
b - tranquilo.
8. a- inovador.
b - tradicional.
9. a- verdadeiro, franco e direto.
b - de por “panos quentes”.
10. a - impulsivo.
b - hesitante
11. a - inventivo.
rotineiro
12. a - justo.
b - misericordioso.
13. a - um aventureiro.
b - um pesquisador.
14. a - improvisador.
b - estrategista.
15. a - impessoal e lógico.
b - pessoal e sentimental.
16. a - disperso.
b- compenetrado.
17.a- imprevisível.
b- previsível.
18. a - consistente e razoável nas suas idéias.
b - profundo nos seus sentimentos.





In [30]:
question_number = 1
questionario = {}
headers = {}
for i in range(1,len(starts)):
    content = extract_substring(text_content, starts[i], ends[i])
    list_content = content.split(' ---')

    headers[starts[i]]=list_content[0].strip()

    # Regular expression to find the questions (start with a number followed by ".")
    questions_list = re.split(r'(?m)(?=^\d{1,2}\.\s)', list_content[1].strip())
    
    questions = []
    for q in questions_list:  # Assuming 'questions_list' contains the list of questions
        if q.strip():  # Only increment if the string is not empty
            q_cleaned = q.split('.', 1)[1].strip()
            q_cleaned = re.sub(r'(?<!\w)a\s*-\s*', '', q_cleaned).strip()

            options_list = re.split(r'b\s*-\s*', q_cleaned)
            options_list.append('Nenhuma ou ambas.')

            questions.append(('Q' + str(question_number), options_list))
            question_number += 1

    questionario[starts[i]] = dict(questions)

In [34]:
[m+opt for m,opt in zip(['a - ', 'b - ', ''], options_list)]

['a - geralmente critica as idéias de seus colegas.\n',
 'b - geralmente critica seus colegas.',
 'Nenhuma ou ambas.']